In [88]:
import pexpect
import json
from os.path import basename, splitext, dirname, join, exists
from glob import glob
from pytezos.michelson.parse import MichelsonParser, michelson_to_micheline

In [91]:
project_dir = dirname(globals()['_dh'][0])
tzt_dir = '/tmp/tzt'

In [92]:
if not exists(tzt_dir):
    pexpect.run(f'git clone https://gitlab.com/tezos/tzt-reference-test-suite.git {tzt_dir}')

In [64]:
template = """
mod runner;

use proto::Result;
use runner::tzt::TZT;

const CASE: &str = "
{tzt}";

#[test]
fn {name}() -> Result<()> {{
    TZT::try_from(CASE)?.run()
}}
"""

In [94]:
ignore = [
    '/tmp/tzt/setdelegate_00.tzt',
    '/tmp/tzt/transfertokens_00.tzt',
    '/tmp/tzt/transfertokens_01.tzt',
    '/tmp/tzt/createcontract_00.tzt',
    '/tmp/tzt/createcontract_01.tzt',
]
# TODO: replace '_' with something that can be parsed with the current grammar

In [95]:
parser = MichelsonParser(extra_primitives=[
    'Contract', 'other_contracts', 'GeneralOverflow', 'Failed', 'MutezUnderflow', 'MutezOverflow'
])
for filename in glob(f'{tzt_dir}/*.tzt'):
    # print(filename)
    if filename in ignore:
        continue
    with open(filename) as f:
        name = 'tzt_' + splitext(basename(filename))[0].replace('-', '_')
        out = join(project_dir, 'proto', 'tests', f'{name}.rs')
        if not exists(out):
            src = michelson_to_micheline(f.read(), parser=parser)
            tzt = json.dumps(src, indent=2).replace('\"', '\\"')
            dst = template.format(tzt=tzt, name=name)
            with open(out, 'w') as rs:
                rs.write(dst)